If you want to be in the `PedestrianAvoidancePOMDP` environment, run the following cell. If the package has been added to the main julia environment, it is not necessary.

In [1]:
] activate ../ 

In [8]:
using Revise
using AutomotiveDrivingModels
using AutoViz
using AutomotiveSensors
using AutomotivePOMDPs
using GridInterpolations

using PedestrianAvoidancePOMDP
using POMDPPolicies

using JLD2
using Reel
using Random

In [2]:
policy = load("../policy/policy.jld2")["policy"]; # need to be updated

In [9]:
pomdp = SingleOCFPOMDP()
updater = SingleOCFUpdater(pomdp)

parameters = CrosswalkParams()
parameters.roadway_length = 400.0
parameters.obstacles_visible = false

obstacle_offset = -2
obstacle_1 = ConvexPolygon([VecE2(34, obstacle_offset), VecE2(34, obstacle_offset-3), VecE2(46.5, obstacle_offset-3), VecE2(46.5, obstacle_offset)],4)
obstacle_2 = ConvexPolygon([VecE2(34, +4.5), VecE2(34, +7.5), VecE2(46.5, +7.5), VecE2(46.5, +4.5)],4)
parameters.obstacles = [obstacle_1, obstacle_2]
#parameters.obstacles = [obstacle_2]

env = CrosswalkEnv(parameters)
pomdp.env = env


ego_id = 1
ped_id = 2
ped2_id = 3


ego_v = 11.11
ped_v = 5.4/3.6
hitpoint = 50             # 50: middle, 100: left corner, 0: right corner
ped_theta = π/2


# fix values
ped_x = 50.0
ped_y_start = -5.0
ego_y = 0.0


ped_y_offset_hitpoint = VehicleDef().width * hitpoint / 100 - VehicleDef().width/2
ped_y_end = ped_y_offset_hitpoint
ped_t_collision = (ped_y_end - ped_y_start) / ped_v; 
ego_x = ped_x - ego_v * ped_t_collision - VehicleDef().length/2;

if ( false )
    ego_v = 0.0
    ego_x = 20
    ped_x = 30
    ped_y_start = -2.0
    ped_v = 0.0
end

ped_y_start = -5.0
ped_v = 2.0

# Car definition
ego_initial_state = VehicleState(VecSE2(ego_x, ego_y, 0.), env.roadway.segments[1].lanes[1], env.roadway, ego_v)
ego = Vehicle(ego_initial_state, VehicleDef(), 1)

# Pedestrian definition 
ped_initial_state = VehicleState(VecSE2(ped_x,ped_y_start,ped_theta), env.crosswalk, env.roadway, ped_v)
ped = Vehicle(ped_initial_state, AutomotivePOMDPs.PEDESTRIAN_DEF, 2)

ped2_v = 0.0
ped2 = Vehicle(VehicleState(VecSE2(54., 5., 0.78), env.crosswalk, env.roadway, ped2_v), AutomotivePOMDPs.PEDESTRIAN_DEF, ped2_id)


scene = Scene()
push!(scene, ego)
push!(scene, ped)
push!(scene, ped2)



cam = FitToContentCamera(0.);
timestep = 0.2


pos_noise = 0.001
vel_noise = 0.000
false_positive_rate = 0.00
false_negative_rate = 0.00
rng = MersenneTwister(1);
sensor = AutomotiveSensors.GaussianSensor(AutomotiveSensors.LinearNoise(10, pos_noise, 0.00), 
                 AutomotiveSensors.LinearNoise(10, vel_noise, 0.00), false_positive_rate, false_negative_rate, rng) 



# define a model for each entities present in the scene
models = Dict{Int, DriverModel}()
models[ego_id] = FrenetPedestrianPOMDP(a=LatLonAccel(0.0, 0.0),
    env=env,
    sensor=sensor,
    obstacles=env.obstacles,
    timestep=timestep,
    pomdp=pomdp,
   # policy=policy,
    updater=updater,
    ego_vehicle=ego,
    desired_velocity=ego_v,
    b=PedestrianAvoidancePOMDP.initBeliefAbsentPedestrian(pomdp, ego_y, ego_v)
)
models[ego_id].pomdp.desired_velocity = ego_v
models[ego_id].pomdp.ΔT = timestep

models[ped_id] = ConstantPedestrian(v_desired=ped.state.v, dawdling_amp=0.001) # dumb model
models[ped2_id] = ConstantPedestrian(v_desired=ped2.state.v, dawdling_amp=0.001) # dumb model



nticks = 50
rec = SceneRecord(nticks+1, timestep)

risk = Float64[]
sensor_observations = [Vehicle[]]
belief = Dict{Int64, SingleOCFBelief}[]
ego_vehicle = Vehicle[]
action_pomdp = SingleOCFAction[]

obs_callback = (ObservationCallback(risk,sensor_observations, belief, ego_vehicle, action_pomdp),)

tic()
simulate!(rec, scene, env.roadway, models, nticks, obs_callback)
toc()




MethodError: MethodError: no method matching actionindex(::SingleOCFPOMDP, ::PedestrianAvoidancePOMDP.SingleOCFAction)
Closest candidates are:
  actionindex(!Matched::POMDPModelTools.FullyObservablePOMDP{S,A}, ::A) where {S, A} at /home/xubuntu/.julia/packages/POMDPModelTools/zVVz4/src/fully_observable_pomdp.jl:45
  actionindex(!Matched::POMDPModelTools.UnderlyingMDP{P,S,A}, ::A) where {P, S, A} at /home/xubuntu/.julia/packages/POMDPModelTools/zVVz4/src/underlying_mdp.jl:31

In [4]:
# Visualize scenario
duration, fps, render_hist = animate_record(rec, timestep, env, sensor, sensor_observations, risk, belief, ego_vehicle, action_pomdp, CarFollowCamera(2,10.0))
film = roll(render_hist, fps = fps, duration = duration)

Reel.Frames{MIME{Symbol("image/png")}}("/tmp/tmpNNn5fO", 0x0000000000000010, 5.0, nothing)

In [23]:
x = findmin([6 4 3])

(3, CartesianIndex(1, 3))

In [46]:
x = PedestrianAvoidancePOMDP.initBeliefAbsentPedestrian(pomdp, 1., 2.)

POMDPModelTools.SparseCat{Array{PedestrianAvoidancePOMDP.SingleOCFState,1},Array{Float64,1}}(PedestrianAvoidancePOMDP.SingleOCFState[[1.0, 2.0, -10.0, -10.0, 0.0, 0.0]], [0.2])

In [17]:
obs = PedestrianAvoidancePOMDP.SingleOCFState(0., 1.1, 10.,1., 1.57, 0.)
obs_int = pomdp.state_space[PedestrianAvoidancePOMDP.state_index(pomdp, obs)]

6-element PedestrianAvoidancePOMDP.SingleOCFState:
  0.0 
  2.0 
 10.0 
  1.0 
  1.57
  0.0 

In [9]:
s = PedestrianAvoidancePOMDP.SingleOCFState(0., 1.5, 1.0, 0., 0., 0.)

6-element PedestrianAvoidancePOMDP.SingleOCFState:
 0.0
 1.5
 1.0
 0.0
 0.0
 0.0

In [47]:
x

POMDPModelTools.SparseCat{Array{PedestrianAvoidancePOMDP.SingleOCFState,1},Array{Float64,1}}(PedestrianAvoidancePOMDP.SingleOCFState[[1.0, 2.0, -10.0, -10.0, 0.0, 0.0]], [0.2])

In [89]:
id_tmp

([9758, 9763, 9833, 9838], [0.25, 0.25, 0.25, 0.25])

In [91]:
id = findmax(id_tmp[1])[1]

9838

In [41]:
probs = ones(10)
probs[1:end-1] .= 20 / length(5)
probs[end] = 19


19

In [42]:
probs

10-element Array{Float64,1}:
 20.0
 20.0
 20.0
 20.0
 20.0
 20.0
 20.0
 20.0
 20.0
 19.0

UndefVarError: UndefVarError: state_index not defined

In [7]:
collect(LinRange(-0.39/2., 0.39/2., 3))

3-element Array{Float64,1}:
 -0.195
  0.0  
  0.195